https://twitter.com/yujitach/status/1424030835771023363

In [1]:
using LinearAlgebra
using LoopVectorization
using LinearMaps
using ArnoldiMethod

function prepareDiag(L)
    diag = zeros(2^L)
    for state = 1:2^L
        for i = 1:L
            j = i==L ? 1 : i+1
            @inbounds diag[state] -= (((state >> (i-1))&1) == ((state >> (j-1))&1)) ? 1 : -1
        end
    end
    diag
end
    
function Hfunc!(C, B, diag, L)
    N = length(diag)
    @tturbo for state = 1:N
        C[state] = diag[state] * B[state]
    end
    for i = 1:L
        @tturbo for state = 1:N
            newstate = (state&(~(2^L))) ⊻ (1<<(i-1))
            c = newstate == 0
            newstate = !c*newstate + c*N # remove if statement
            C[newstate] -= B[state]
        end
    end
end
prepareHfunc!(L) = (C, B) -> Hfunc!(C, B, prepareDiag(L), L)

function f_LinearMap(L)
    H_LinearMap = LinearMap(prepareHfunc!(L), 2^L, ismutating=true, issymmetric=true, isposdef=false)
    d_LM, h_LM = partialschur(H_LinearMap, nev=1, which=SR())
    d_LM.eigenvalues[1]
end

f_exact(L) = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)

f_LinearMap(20), f_exact(20)

(-25.490989686364657 + 0.0im, -25.49098968636475)

In [2]:
@time f_LinearMap(20)
@time f_LinearMap(20)
@time f_LinearMap(20)

  6.998082 seconds (614 allocations: 968.031 MiB, 0.47% gc time)
  7.010990 seconds (618 allocations: 968.031 MiB, 1.15% gc time)
  7.056832 seconds (621 allocations: 968.032 MiB, 0.82% gc time)


-25.490989686364767 + 0.0im

https://juliaphysics.github.io/PhysicsTutorials.jl/tutorials/general/quantum_ising/quantum_ising.html

In [3]:
using LinearAlgebra
using SparseArrays
using ArnoldiMethod
⊗(x,y) = kron(x,y)

function TransverseFieldIsing_sparse(;N,h)
    id = [1 0; 0 1] |> sparse
    σˣ = [0 1; 1 0] |> sparse
    σᶻ = [1 0; 0 -1] |> sparse
    
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    H = spzeros(Int, 2^N, 2^N)
    for i in 1:N
        H -= foldl(⊗, first_term_ops)
        first_term_ops = circshift(first_term_ops,1)
    end
    
    for i in 1:N
        H -= h*foldl(⊗, second_term_ops)
        second_term_ops = circshift(second_term_ops,1)
    end
    H
end

function f_sparse(L)
    H = TransverseFieldIsing_sparse(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

f_exact(L) = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)

f_sparse(20), f_exact(20)

(-25.49098968636474 + 0.0im, -25.49098968636475)

In [4]:
L = 20
@time H = TransverseFieldIsing_sparse(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.954490 seconds (4.75 k allocations: 6.718 GiB, 15.25% gc time, 0.28% compilation time)
  6.425090 seconds (246 allocations: 328.026 MiB, 0.43% gc time, 0.21% compilation time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.23608646331041197; -0.06481741688439337; … ; -0.06481741680178926; -0.23608646323348847;;], [-25.49098968636478;;], ComplexF64[-25.49098968636478 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [5]:
L = 20
@time H = TransverseFieldIsing_sparse(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  3.025552 seconds (4.66 k allocations: 6.718 GiB, 18.19% gc time)
  6.172042 seconds (70 allocations: 328.017 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646320973226; 0.06481741675939662; … ; 0.06481741678115605; 0.23608646324381535;;], [-25.490989686364703;;], ComplexF64[-25.490989686364703 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [6]:
L = 20
@time H = TransverseFieldIsing_sparse(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  3.009844 seconds (4.66 k allocations: 6.718 GiB, 17.19% gc time)
  6.855470 seconds (71 allocations: 328.017 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646305480757; 0.06481741675765908; … ; 0.06481741682791009; 0.23608646341856734;;], [-25.490989686364745;;], ComplexF64[-25.490989686364745 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [7]:
TransverseFieldIsing_sparse(N=20, h=1)

1048576×1048576 SparseMatrixCSC{Int64, Int64} with 21650584 stored entries:
⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿

In [8]:
function TransverseFieldIsing_sparse_revised(;N, h)
    id = [1 0; 0 1] |> sparse
    σˣ = [0 1; 1 0] |> sparse
    σᶻ = [1 0; 0 -1] |> sparse
    
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    tmp = [spzeros(Int, 2^k, 2^k) for k in 1:N]
    H = spzeros(Int, 2^N, 2^N)
    
    for i in 1:N
        tmp[1] .= first_term_ops[mod1(i, N)]
        for k in 2:N
            kron!(tmp[k], tmp[k-1], first_term_ops[mod1(i+k-1, N)])
        end
        H .-= tmp[N]
    end
    
    for i in 1:N
        tmp[1] .= second_term_ops[mod1(i, N)]
        for k in 2:N
            kron!(tmp[k], tmp[k-1], second_term_ops[mod1(i+k-1, N)])
        end
        H .-= h .* tmp[N]
    end
    H
end

function f_sparse_revised(L)
    H = TransverseFieldIsing_sparse_revised(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

TransverseFieldIsing_sparse_revised(N=10, h=1) == TransverseFieldIsing_sparse(N=10, h=1)

true

In [9]:
L = 20
@time H = TransverseFieldIsing_sparse_revised(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.634633 seconds (385 allocations: 4.078 GiB, 14.19% gc time)
  6.180826 seconds (67 allocations: 328.016 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646329398694; 0.06481741671479442; … ; 0.06481741677502564; 0.23608646326086227;;], [-25.49098968636457;;], ComplexF64[-25.49098968636457 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [10]:
L = 20
@time H = TransverseFieldIsing_sparse_revised(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.639796 seconds (385 allocations: 4.078 GiB, 13.87% gc time)
  8.812041 seconds (68 allocations: 328.016 MiB, 1.75% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.2360864634510118; 0.06481741680576163; … ; 0.0648174167763798; 0.2360864630405447;;], [-25.490989686364745;;], ComplexF64[-25.490989686364745 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [11]:
L = 20
@time H = TransverseFieldIsing_sparse_revised(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.457894 seconds (385 allocations: 4.078 GiB, 6.30% gc time)
  6.303066 seconds (68 allocations: 328.017 MiB, 1.39% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646321942076; 0.06481741682882682; … ; 0.06481741676495698; 0.23608646324728416;;], [-25.490989686364763;;], ComplexF64[-25.490989686364763 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [12]:
TransverseFieldIsing_sparse_revised(N=20, h=1)

1048576×1048576 SparseMatrixCSC{Int64, Int64} with 21650584 stored entries:
⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿

In [13]:
bit(a, k) = (a >> k) & 1

function sigmaz2(N)
    d = zeros(Int, 2^N)
    for k in 1:N, a in 0:2^N-1
        d[a+1] -= ifelse(bit(a, N-k) == bit(a, N - mod1(k+1, N)), 1, -1)
    end
    d
end

function sigmax(N, k)
    v = zeros(Int, 2^N - 2^(N-k))
    for i in 1:2:2^k-1
        v[(2^(N-k)*(i-1) + 1):2^(N-k)*i] .= -1
    end
    v
end

function TransverseFieldIsing_sparse_revised2(;N, h)
    d = sigmaz2(N)    
    v = sigmax.(N, 1:N)    
    H = spdiagm(
        (-2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        ( 2^(N-k) => v[k] for k in 1:N)...)
end

function f_sparse_revised2(L)
    H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

TransverseFieldIsing_sparse_revised2(N=10, h=1) == TransverseFieldIsing_sparse(N=10, h=1)

true

In [14]:
L = 20
@time H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.117366 seconds (21.90 k allocations: 2.314 GiB, 0.64% gc time, 0.33% compilation time)
  8.971829 seconds (66 allocations: 328.016 MiB, 1.25% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.2360864633854531; -0.06481741687207625; … ; -0.06481741678547956; -0.23608646316295315;;], [-25.490989686364735;;], ComplexF64[-25.490989686364735 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [15]:
L = 20
@time H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.472227 seconds (485 allocations: 2.313 GiB, 14.28% gc time)
  9.172846 seconds (66 allocations: 328.016 MiB, 2.44% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.23608646289610324; -0.0648174166615296; … ; -0.06481741685765068; -0.2360864635797589;;], [-25.490989686364703;;], ComplexF64[-25.490989686364703 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [16]:
L = 20
@time H = TransverseFieldIsing_sparse_revised2(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  2.429283 seconds (485 allocations: 2.313 GiB, 13.51% gc time)
  9.201822 seconds (70 allocations: 328.017 MiB, 2.28% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646346807252; 0.06481741691576823; … ; 0.06481741667368268; 0.23608646309026252;;], [-25.49098968636476;;], ComplexF64[-25.49098968636476 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [17]:
function TransverseFieldIsing_sparse_revised2_time(;N, h)
    @time d = sigmaz2(N)    
    @time v = sigmax.(N, 1:N)    
    @time H = spdiagm(
        (-2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        ( 2^(N-k) => v[k] for k in 1:N)...)
end

TransverseFieldIsing_sparse_revised2_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised2_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised2_time(N=20, h=1)

  0.117869 seconds (2 allocations: 8.000 MiB)
  0.066151 seconds (41 allocations: 152.001 MiB, 40.96% gc time)
  2.234131 seconds (11.41 k allocations: 2.157 GiB, 13.51% gc time, 0.96% compilation time)

  0.116526 seconds (2 allocations: 8.000 MiB)
  0.040681 seconds (41 allocations: 152.001 MiB)
  2.486643 seconds (440 allocations: 2.156 GiB, 17.64% gc time)

  0.116804 seconds (2 allocations: 8.000 MiB)
  0.038584 seconds (41 allocations: 152.001 MiB)
  2.430211 seconds (440 allocations: 2.156 GiB, 19.43% gc time)


1048576×1048576 SparseMatrixCSC{Int64, Int64} with 40894466 stored entries:
⣿⣿⣾⣦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠻⣿⣿⣿⣿⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠻⡿⣿⣿

In [18]:
bit(a, k) = (a >> k) & 1

function sigmaz2_sparse(N)
    d = zeros(Int, 2^N)
    for k in 1:N, a in 0:2^N-1
        d[a+1] -= ifelse(bit(a, N-k) == bit(a, N - mod1(k+1, N)), 1, -1)
    end
    sparse(d) # should be sparse
end

function sigmax_sparse(N, k)
    v = zeros(Int, 2^N - 2^(N-k))
    for i in 1:2:2^k-1
        v[(2^(N-k)*(i-1) + 1):2^(N-k)*i] .= -1
    end
    sparse(v) # should be sparse
end

function TransverseFieldIsing_sparse_revised3(;N, h)
    d = sigmaz2_sparse(N)    
    v = sigmax_sparse.(N, 1:N)    
    H = spdiagm(
        (-2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        ( 2^(N-k) => v[k] for k in 1:N)...)
end

function f_sparse_revised3(L)
    H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

TransverseFieldIsing_sparse_revised3(;N=10, h=1) == TransverseFieldIsing_sparse(;N=10, h=1)

true

In [19]:
L = 20
@time H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  1.885320 seconds (22.24 k allocations: 1.948 GiB, 25.43% gc time, 0.46% compilation time)
  6.144846 seconds (68 allocations: 328.016 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646342098635; 0.06481741685044573; … ; 0.06481741672015905; 0.23608646302451014;;], [-25.490989686364717;;], ComplexF64[-25.490989686364717 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [20]:
L = 20
@time H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  1.864288 seconds (859 allocations: 1.947 GiB, 24.79% gc time)
  6.913462 seconds (69 allocations: 328.017 MiB, 2.07% gc time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.2360864633332615; 0.06481741679110317; … ; 0.06481741680489875; 0.23608646312630016;;], [-25.490989686364678;;], ComplexF64[-25.490989686364678 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [21]:
L = 20
@time H = TransverseFieldIsing_sparse_revised3(N=L, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  1.738984 seconds (859 allocations: 1.947 GiB, 20.36% gc time)
  6.133689 seconds (69 allocations: 328.017 MiB)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([-0.23608646323858584; -0.06481741682626445; … ; -0.06481741681770777; -0.23608646331891392;;], [-25.49098968636463;;], ComplexF64[-25.49098968636463 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)

In [22]:
function TransverseFieldIsing_sparse_revised3_time(;N, h)
    @time d = sigmaz2_sparse(N)    
    @time v = sigmax_sparse.(N, 1:N)    
    @time H = spdiagm(
        (-2^(N-k) => v[k] for k in 1:N)...,
        0 => d,
        ( 2^(N-k) => v[k] for k in 1:N)...)
end

TransverseFieldIsing_sparse_revised3_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised3_time(N=20, h=1); println()
TransverseFieldIsing_sparse_revised3_time(N=20, h=1)

  0.128349 seconds (12 allocations: 25.879 MiB)
  0.334002 seconds (241 allocations: 456.943 MiB, 52.55% gc time)
  1.435743 seconds (11.56 k allocations: 1.476 GiB, 19.08% gc time, 1.64% compilation time)

  0.124450 seconds (12 allocations: 25.879 MiB)
  0.407541 seconds (241 allocations: 456.943 MiB, 61.91% gc time)
  1.289654 seconds (604 allocations: 1.475 GiB, 13.74% gc time)

  0.120620 seconds (12 allocations: 25.879 MiB)
  0.404538 seconds (241 allocations: 456.943 MiB, 62.34% gc time)
  1.329586 seconds (604 allocations: 1.475 GiB, 13.98% gc time)


1048576×1048576 SparseMatrixCSC{Int64, Int64} with 21650584 stored entries:
⣿⣿⣾⢦⡀⠳⣄⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⡀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠙⢦⡈⠳⣼⣿⣿⡆⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠙⢦⡀⠀⠀⠁⠀⠈⠈⠉⣿⣿⣾⢦⡀⠳⣄⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠺⣟⢻⣶⣿⡂⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢤⡈⠻⠻⠿⣧⣤⣠⡈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠀⣻⣿⣿⣙⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡈⠳⣼⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡟⢦⡈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠙⢦⠈⠳⡿⣿⣿⣀⡀⡀⠀⢀⠀⠀⠈⠳⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠸⣿⣿⡟⢦⡈⠳⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠈⠻⣍⣿⣿⣯⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠐⢦⡈⠋⠛⢻⣶⣦⣦⡈⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢦⡀⠨⣿⠿⣧⣽⡦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠙⢦⠈⠳⡿⣿⣿